In [20]:
import pandas as pd
import numpy as np
import random

In [30]:
num_cells = 800

In [ ]:
freqs = [0 for i in range(num_cells)]
initial = 1.25
final = 20
freq_step = 0.005

for i in range(num_cells):
    if i == 0:
        freqs[i] = initial
        continue

    freqs[i] = freqs[i-1] + freqs[i-1]*2*freq_step
    
temp = 5


In [ ]:
conn_params = {'BW': {'AN_W': 2,
                      'AN_I2': 0.4,
                      'AN_P': 0.4,
                      'W_I2': 0.1,
                      'W_P': 0.1,
                      'I2_P': 0.6},
               'N': {'AN_W': 140,
                     'AN_I2': 48,
                     'AN_P': 48,
                     'W_I2': 15,
                     'W_P': 15,
                     'I2_P': 21},
               'strength': {'AN_W': 0.06,
                            'AN_I2': 0.55,
                            'AN_P': 0.25,
                            'W_I2': 1.4,
                            'W_P': 0.1,
                            'I2_P': 2.25}}

df = pd.DataFrame.from_dict(conn_params)


In [5]:
df

,BW,N,strength
AN_W,2.0,140,0.06
AN_I2,0.4,48,0.55
AN_P,0.4,48,0.25
W_I2,0.1,15,1.40
W_P,0.1,15,0.10
I2_P,0.6,21,2.25


In [ ]:
cell_ids = {'AN': [i for i in range(num_cells)],
            'W': [i for i in range(num_cells)],
            'I2': [i for i in range(num_cells)],
            'P': [i for i in range(num_cells)]}

conns = {'AN_W': [],
         'AN_I2': [],
         'AN_P': [],
         'W_I2': [],
         'W_P': [],
         'I2_P': []}

for conn, row in df.iterrows():
    temp = 5

    source = conn.split('_')[0]
    source_ids = cell_ids[source]

    target = conn.split('_')[1]
    target_ids = cell_ids[target]

    conn_list = []

    for target_id in target_ids:

        source_BW = row.BW / freq_step
        num_source = int(row.N)

        ub = int(target_id + source_BW/2)
        lb = int(target_id - source_BW/2)
        if lb < 0: lb = 0

        source_pool = source_ids[lb:ub]
        try:
            source_rand = random.sample(source_pool,num_source)
        except ValueError:
            print(conn,target_id)

        conn_list.extend([[i,target_id] for i in source_rand])

    conns[conn] = conn_list


AN_I2 0
AN_I2 1
AN_I2 2
AN_I2 3
AN_I2 4
AN_I2 5
AN_I2 6
AN_I2 7
AN_I2 793
AN_I2 794
AN_I2 795
AN_I2 796
AN_I2 797
AN_I2 798
AN_I2 799
AN_P 0
AN_P 1
AN_P 2
AN_P 3
AN_P 4
AN_P 5
AN_P 6
AN_P 7
AN_P 793
AN_P 794
AN_P 795
AN_P 796
AN_P 797
AN_P 798
AN_P 799
W_I2 0
W_I2 1
W_I2 2
W_I2 3
W_I2 4
W_I2 796
W_I2 797
W_I2 798
W_I2 799
W_P 0
W_P 1
W_P 2
W_P 3
W_P 4
W_P 796
W_P 797
W_P 798
W_P 799
